# AA1 lab 07

# Linear classification

In [ ]:
# Uncomment to upgrade packages
# !pip install pandas --upgrade --user --quiet
# !pip install numpy --upgrade --user --quiet
# !pip install scipy --upgrade --user --quiet
# !pip install statsmodels --upgrade --user --quiet
# !pip install scikit-learn --upgrade --user --quiet
# !pip install tensorflow --user
%load_ext autoreload

In [ ]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

import matplotlib.pyplot as plt
import seaborn as sns

from time import time
from datetime import timedelta

from sklearn.model_selection import (
    train_test_split,
    KFold,
    cross_validate,
    GridSearchCV,
)
from sklearn.preprocessing import MinMaxScaler
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis,
    QuadraticDiscriminantAnalysis,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB, CategoricalNB
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression

from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
)

from IPython.core.interactiveshell import InteractiveShell

pd.set_option("display.precision", 3)

In [ ]:
import warnings

warnings.filterwarnings("ignore")

np.random.seed(123)  # for reproducibility

### Support functions

In [ ]:
def confusion(true, pred):
    """
    Function for pretty printing confusion matrices
    """
    true.name = "target"
    pred.name = "predicted"
    cm = pd.crosstab(true.reset_index(drop=True), pred.reset_index(drop=True))
    cm = cm[cm.index]
    return cm

## SECTION 1. Linear Classification on wine data

 We have the results of an analysis on wines grown in a region in Italy but derived from three different cultivars.
The analysis determined the quantities of 13 chemical constituents found in each of the three types of wines. 
The goal is to separate the three types of wines. 

For increasing the interest on the exercise we will asume that one of the wine types is cheap, the second is normal, and the third is expensive. Our objective here will be to make a classifier that labels the wine its category price. 

In [ ]:
from sklearn.datasets import load_wine

data = load_wine(as_frame=True)

df = data.frame
df["target"] = df["target"].map({0: "cheap", 1: "normal", 2: "expensive"})
df.target = df.target.astype(
    CategoricalDtype(categories=["cheap", "normal", "expensive"], ordered=True)
)
print(df.shape)
df.head()

If we do a fast pairplot we can see that this dataset is going to be an easy one. The scatterplots are showing clear separation on some variables, and it looks like we would be able to predict the class with only a subset on the variables, like for example Alcohol and OD280/OD315.

In [ ]:
sns.pairplot(data=df, hue="target");

### Resampling protocol

We will use two data partitions (train and test) with 5-fold cross-validation over the train partition for deciding hyperparameters. 
For comparing models we will use 5-fold cross-validation metrics.

We have chosen this strategy because we have very few samples (178) and we are going to use quite fast models on this lab. 

In [ ]:
X = df.loc[:, df.columns != "target"]
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

### Preprocessing

Our variables are showing different ranges, we are going to scale them because we don't want this to interfere with our models' performance. *Notice that the scaling process is different for the training and the test set; this is to make things compatible with the fact that we cannot use information from the test set to do the scaling*

In [ ]:
def preprocessing(X, y, scaler=None):
    # We scale all the columns
    if scaler is None:
        # We only want the scaler to fit the train data
        scaler = MinMaxScaler()
        X = scaler.fit_transform(X)
    else:
        X = scaler.transform(X)
    return X, y, scaler


X_train, y_train, scaler = preprocessing(X_train, y_train)
X_test, y_test, _ = preprocessing(X_test, y_test, scaler)

### LDA

LDA tries to model the probability $p(y=C_k|X=x)$ by assuming: 
* $p(x|C_k)$ is Gaussian (which means that can be described by $\mu_k$ and $\Sigma_k$)
* All covariance matrix are the same ($\Sigma_k = \Sigma$)

By using bayes formula ($p(A|B) = \frac{P(B|A)P(A)}{P(B)}$) and all these asumptions, we obtain the next discriminant function:

$a_k(x) = x^T\Sigma^{-1}\mu_k - \frac{1}{2}\mu_k^T\Sigma^{-1}\mu_k + log(\pi_k)$

Where $\pi_k$ are the prior probabilities. 

If we call:

$w = \Sigma^{-1}\mu_k$

$w_0=- \frac{1}{2}\mu_k^T\Sigma^{-1}\mu_k + log(\pi_k)$

We obtain a linear representation of the formula.

In [ ]:
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X_train, y_train)

print("Priors:", lda_model.priors_)
print("Means:\n")
means = pd.DataFrame(lda_model.means_)
means.columns = wine.columns[1:]
means.index = lda_model.classes_
means
print("Coefs:")
coefs = pd.DataFrame(lda_model.coef_)
coefs.columns = wine.columns[1:]
coefs.index = lda_model.classes_
coefs.T

print("Intercepts:")
intercepts = pd.DataFrame(lda_model.intercept_)
intercepts.index = lda_model.classes_
intercepts

print("Explained Variance Ratio")
pd.DataFrame(lda_model.explained_variance_ratio_)

We can see now an example of how the model is predicting the class of a single sample.

In [ ]:
sample = X_train[0, :]
sample_value = y_train[0]

sample

The model coeficients correspond to the $w$ in our linear formula and the intercept corresponds to the $w_0$. 

We can use them to obtain the decision function values of our sample. We will predict the class with the biggest value.

In [ ]:
np.matmul(lda_model.coef_, sample) + lda_model.intercept_

We can also compute this using the decision function implemented on the lda class.

In [ ]:
lda_model.decision_function(sample.reshape(1, -1))

In this case the prediction would be cheap, as it is the category codified first on the model. 

In [ ]:
lda_model.classes_

We can also use predict method. 

In [ ]:
print(
    "Prediction: {}\nReal value: {}".format(
        lda_model.predict(sample.reshape(1, -1))[0], sample_value
    )
)

A big advantage for the LDA model is that it can also perform dimensionality reduction. We can now check the projection of the data into the first two components.

In [ ]:
y_train

In [ ]:
X_transformed = lda_model.transform(X_train)

X_transformed = pd.DataFrame(X_transformed)
X_transformed["labels"] = y_train.reset_index(drop=True)
X_transformed

It looks like it is separating well the samples. Now let's evaluate our results.

In [ ]:
sns.scatterplot(x=0, y=1, data=X_transformed, hue="labels")

We can see on the confusion matrix that on the train data we are obtaining a perfect classification. 

In [ ]:
confusion(y_train, pd.Series(lda_model.predict(X_train)))

### Choosing our metrics

Now we are going to compute our cross validation metrics. 



Last time we focussed on Accuracy and Recall. This time we are adding other metrics that are comonly used on classification problems. 


**Accuracy:**

$$accuracy = \frac{\sum_c tp_c}{n}$$

Where tp_c are the true positive predictions for all the classes and n are the total number of samples. This metric is **sensitive to imbalanced data**.

**Precision (of a class):**

$$precision_c = \frac{tp}{tp + fp}$$

Where $tp$ are the true positives (samples correctly predicted of this class) and fp are the false positives (samples from another class predicted incorrectly as this class). This metric measures how much the model is predicting correctly a class with respect all the predictions of this class. We will use this metric when having false positive predictions is very harmful in our model context. 


**Recall (of a class):**

$$recall_c = =\frac{tp}{tp + fn}$$

Where $tp$ are the true positives (samples correctly predicted of this class) and fn are the false negatives (samples from this class predicted incorrectly as a different class). This metric measures how much the model is predicting correctly a class with respect all the real values of this class. We will use this metric when having false negative predictions is very harmful in our model context.

**F1-score (of a class):**

$$\frac{2 * precission_c * recall_c }{precission_c + recall_c}$$

The **harmonic mean** of precision and recall. We will use this metric when we want a good balance between precision and recall.  

We are going to use the: Accuracy, F1 macro, precission macro and recall macro metrics. 

These metrics will give us a precise view of how our model is performing. In this particular problem all the categories are equally important, as we don't want either angry customers or angry sellers. For this reason we will use the macro average of our metrics instead of focusing on the metrics of one specific class. The macro average means averaging the class-metrics.

We will use the accuracy metric because classes are more or less balanced. If classes were strongly unbalanced, accuracy would not be a good metric choice, as it could be deceiving. 

In [ ]:
results_df = pd.DataFrame(
    index=[], columns=["Accuracy", "F1 Macro", "Precision Macro", "Recall Macro"]
)

In [ ]:
cross_val_results = pd.DataFrame(
    cross_validate(
        lda_model,
        X_train,
        y_train,
        cv=5,
        scoring=["accuracy", "f1_macro", "precision_macro", "recall_macro"],
    )
)

results_df.loc["LDA", :] = (
    cross_val_results[
        ["test_accuracy", "test_f1_macro", "test_precision_macro", "test_recall_macro"]
    ]
    .mean()
    .values
)
results_df

###  QDA

QDA is very similar to LDA.  The main difference is that in this model we do not assume that all the classes have the same covariance. This leads to obtaining a quadratic decision surface. This model can also be regularized with its regularization parameter.

In [ ]:
qda_model = QuadraticDiscriminantAnalysis(reg_param=0.1).fit(X_train, y_train)

print("Priors:", qda_model.priors_)
print("Means:\n")
means = pd.DataFrame(qda_model.means_)
means.columns = wine.columns[1:]
means

Now the prediction on the train data is _almost_ perfect. There are three normal wines that the model confused.

In [ ]:
confusion(y_train, pd.Series(qda_model.predict(X_train)))

If we compute again or cv metrics we can see that there is not a lot of difference but LDA is wining. That might be because our dataset is too easy, which makes it difficult to compare the models (since all of them do very well). *Remember this when you chose your dataset for the final project*

In [ ]:
cross_val_results = pd.DataFrame(
    cross_validate(
        qda_model,
        X_train,
        y_train,
        cv=5,
        scoring=["accuracy", "f1_macro", "precision_macro", "recall_macro"],
    )
)

results_df.loc["QDA", :] = (
    cross_val_results[
        ["test_accuracy", "test_f1_macro", "test_precision_macro", "test_recall_macro"]
    ]
    .mean()
    .values
)
results_df

### $k$-NN

$k$-nn stores the training set samples and uses them to classify the new ones. Each new sample will go to the class with the most similar values from the train set (in terms of a specific distance).

The $k$ value tells how many samples will be used to compare. 

We are going to use gridsearch cross-validation to decide which one is the best $k$ and the best distance metric for our data. 

In [ ]:
knn = KNeighborsClassifier()

knn_cv = GridSearchCV(
    estimator=knn,
    param_grid={
        "n_neighbors": [1, 3, 5, 7, 10, 15, 20],
        "metric": ["euclidean", "minkowski", "manhattan"],
    },
    scoring=["accuracy", "f1_macro", "precision_macro", "recall_macro"],
    refit=False,
)

knn_cv.fit(X_train, y_train)
results_cv = pd.DataFrame(knn_cv.cv_results_)

In [ ]:
results_cv.columns

In [ ]:
cols = [
    "param_n_neighbors",
    "param_metric",
    "mean_test_accuracy",
    "mean_test_f1_macro",
    "mean_test_precision_macro",
    "mean_test_recall_macro",
    "std_test_accuracy",
    "std_test_f1_macro",
    "std_test_precision_macro",
    "std_test_recall_macro",
]
results_cv[cols].sort_values(by="mean_test_f1_macro", ascending=False)

It looks like the parameters don't affect the model performance by much this time. We will take $k$=7 and distance = 'minkowski' and add $k$-nn to our results table.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7, metric="minkowski")
knn.fit(X_train, y_train)

cross_val_results = pd.DataFrame(
    cross_validate(
        knn,
        X_train,
        y_train,
        cv=5,
        scoring=["accuracy", "f1_macro", "precision_macro", "recall_macro"],
    )
)

results_df.loc["KNN", :] = (
    cross_val_results[
        ["test_accuracy", "test_f1_macro", "test_precision_macro", "test_recall_macro"]
    ]
    .mean()
    .values
)
results_df

### Gaussian Naive Bayes

The next model we are going to use is Gaussian Naive Bayes. 

Naive Bayes assumes that the attributes of the class conditional probabilities are independent and a certain distribution for them based on the kind of data we are working with.

For example: if we are working with numerical variables it will assume that the features are conditionally independent between them and that they follow a Gaussian distribution.

In [ ]:
gaussian_nb = GaussianNB()

gaussian_nb.fit(X_train, y_train)


cross_val_results = pd.DataFrame(
    cross_validate(
        gaussian_nb,
        X_train,
        y_train,
        cv=5,
        scoring=["accuracy", "f1_macro", "precision_macro", "recall_macro"],
    )
)

results_df.loc["Gaussian Naive Bayes", :] = (
    cross_val_results[
        ["test_accuracy", "test_f1_macro", "test_precision_macro", "test_recall_macro"]
    ]
    .mean()
    .values
)
results_df

### Logistic regression

Finally, we apply logistic regression to see if it improves performance over the previous models.
We use the regularized version, and will optimize its regularization parameter 'C' using cross-validation as usual.


In [ ]:
logreg = LogisticRegressionCV(
    Cs=20, random_state=1, cv=10, scoring="accuracy", multi_class="multinomial"
)

logreg.fit(X_train, y_train)

In [ ]:
# figure out optimal parameter value for 'C'

avg_crossval_scores = logreg.scores_["normal"].mean(axis=0)
idx = np.argmax(avg_crossval_scores)
best_C = logreg.Cs_[idx]
print(best_C)

Let us use the optimal C found in the cross-val metrics for our results table

In [ ]:
logreg = LogisticRegression(C=best_C, multi_class="multinomial")
cross_val_results = pd.DataFrame(
    cross_validate(
        logreg,
        X_train,
        y_train,
        cv=5,
        scoring=["accuracy", "f1_macro", "precision_macro", "recall_macro"],
    )
)

results_df.loc["Logistic Regression", :] = (
    cross_val_results[
        ["test_accuracy", "test_f1_macro", "test_precision_macro", "test_recall_macro"]
    ]
    .mean()
    .values
)

results_df.sort_values(by="Accuracy", ascending=False)

Our results show that Gaussian Naive Bayes is the best model for our wine data, we saw at the beggining that this dataset was extreamly easy, so it makes sense that results are so good and so close. We can also see that the models that have performed better are the most simple ones. 

### See generalization performance

Now we can chose **Logistic regression** as our best model for the wine data. As there are no more decisions to take we can now check the generalization performance of our best model. To ensure that this generalization performance is the closest posible to the performance we would obtain on new data, we are going to use our test partition to evaluate it. 

This time we can see the confusion matrix, as well as the classification report for the test set. This will give us the most detailed possible information about our results.

In [ ]:
y_pred = logreg.fit(X_train, y_train).predict(X_test)

confusion(y_test, pd.Series(y_pred))

print(classification_report(y_test, y_pred))

Our results show that our simple model has been able to predict perfectly our simple test data. Usually, if we obtain perfect results we should be worried, as normally we only obtain them if we make a mistake on the code. But in this particular situation we can belive our results, as we were solving a toy problem.

## SECTION 2. Heart Dataset 
Now we are going to classify a more difficult dataset: Our goal now is to predict whether a patient has a heart disease using medical data (https://www.kaggle.com/ronitf/heart-disease-uci). 

In this context we don't want to miss any patient with the disease, as she might die. We should take this into account when we select our metrics.


In [ ]:
heart = pd.read_csv("heart.csv", delimiter=",")

numerical_cols = ["age", "trestbps", "chol", "thalach", "oldpeak"]

categorical_cols = ["sex", "cp", "exang", "fbs", "restecg", "slope", "ca", "thal"]

target = "target"

heart[categorical_cols] = heart[categorical_cols].astype("category")

heart.describe(include="all")

In [ ]:
heart.columns[:-1]

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(16, 14))

for i, c in enumerate(heart.columns[:-1]):
    ax = axes.reshape(-1)[i]
    if c in categorical_cols:
        a = sns.countplot(x=c, data=heart, ax=ax, hue="target")
    else:
        sns.histplot(x=c, data=heart, hue="target", ax=ax)
plt.tight_layout();

### Resampling protocol

We have a bit more samples that before, so this time we are going to use a train val test strategy. The previous strategy would be a good choice, as we do not have a lot of samples and the models we are using are fast, but we are going to use the train val test strategy instead so you have an example of both. 

In [ ]:
X = heart.loc[:, heart.columns != "target"]
y = heart["target"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=43
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.33, random_state=42
)

### Preprocessing

We have different ranges and categorical variables this time. 
We should handle these two problems in our pre-processing.
Notice the different treatment of trainin set and validation and test sets.

In [ ]:
def preprocessing(X, y, scaler=None):
    # We scale the numerical columns
    if scaler is None:
        # We only want the scaler to fit the train data
        scaler = MinMaxScaler()
        X.loc[:, numerical_cols] = scaler.fit_transform(X[numerical_cols])
    else:
        X.loc[:, numerical_cols] = scaler.transform(X[numerical_cols])
    # We apply one-hot-encoding to the categorical columns
    X = pd.get_dummies(X, drop_first=True)
    return X, y, scaler


X_train, y_train, scaler = preprocessing(X_train, y_train)
X_val, y_val, _ = preprocessing(X_val, y_val, scaler)
X_test, y_test, _ = preprocessing(X_test, y_test, scaler)

### Choosing our metrics

We are going to use the LDA validation results to explain which metrics we are using and why. 

This time we want to avoid missing any patients with a heart condition, so classifying a 1 as a 0 will be very harmful to our model. 

In [ ]:
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X_train, y_train)
y_pred_lda = lda_model.predict(X_val)

In [ ]:
confusion(pd.Series(y_val), pd.Series(y_pred_lda))

If we look into our confusion matrix, we have 2 samples that had a heart disease that where classified as healthy. 
That is a 5% of dead patients (which is unacceptable). We want to select a metric that penalizes this (i.e. will be lower when we predict patients with the disease wrongly). 

So our best choice would be the **recall of the class 1**. If you remember the recall formula: 
$$recall_c =\frac{tp}{tp + fn}$$

A recall of 1 would mean 0 samples predicted as false negative, i.e. cero patients with heart disease predicted as healthy.  

If we chose only this metric, we would risk a model that predicts only class 1, that might not be ideal, as we would need to check all the patients, which could saturate the hospital. To avoid this we will also use the **f1-score of the class 1** as secondary metric. 

It is important to be careful while chosing our metrics and addapt the best we can to the context of our problem, so we can really chose the best model for our context (which might not be the one with the best accuracy).

We will print a set of metrics, not just these ones, but these ones are going to be the ones we will use to take our decisions. 

In [ ]:
print(classification_report(y_val, y_pred_lda))

In [ ]:
def compute_metrics(y_real, y_pred):
    # By default it will compute the binary recall of class 1, we can specify which class do we want by using this parameter
    recall_class_1 = recall_score(y_real, y_pred, pos_label=1)
    f1_class_1 = f1_score(y_real, y_pred, pos_label=1)
    accuracy = accuracy_score(y_real, y_pred)
    f1_macro = f1_score(y_real, y_pred, average="macro")
    precison_macro = precision_score(y_real, y_pred, average="macro")
    recall_macro = recall_score(y_real, y_pred, average="macro")
    return [
        recall_class_1,
        f1_class_1,
        accuracy,
        f1_macro,
        precison_macro,
        recall_macro,
    ]

In [ ]:
results_heart = pd.DataFrame(
    index=[],
    columns=[
        "**Recall class 1**",
        "**F1 class 1**",
        "Accuracy",
        "F1 Macro",
        "Precision Macro",
        "Recall Macro",
    ],
)

results_heart.loc["LDA", :] = compute_metrics(y_val, y_pred_lda)
results_heart

### QDA
Now we are going to train the rest of our models with this new dataset. And store our metrics on our results dataframe. 


In [ ]:
index = pd.MultiIndex.from_arrays([[], []], names=("model", "reg"))
results_qda = pd.DataFrame(
    index=index,
    columns=[
        "**Recall class 1**",
        "**F1 class 1**",
        "Accuracy",
        "F1 Macro",
        "Precision Macro",
        "Recall Macro",
    ],
)

regularization_parameters = [0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1]

for reg in regularization_parameters:
    qda_model = QuadraticDiscriminantAnalysis(reg_param=reg)
    qda_model = qda_model.fit(X_train, y_train)
    y_pred = qda_model.predict(X_val)
    results_qda.loc[("QDA", reg), :] = compute_metrics(y_val, y_pred)

results_qda.sort_values(by="**Recall class 1**", ascending=False)

According to our validation results, the best parameter is 10. We will use this model to compare with the rest.

In [ ]:
qda_model = QuadraticDiscriminantAnalysis(reg_param=0.1)
qda_model = qda_model.fit(X_train, y_train)
y_pred_qda = qda_model.predict(X_val)

results_heart.loc["QDA-0.1", :] = compute_metrics(y_val, y_pred_qda)
results_heart

### KNN 

Lets follow the same process with knn.

In [ ]:
index = pd.MultiIndex.from_arrays([[], []], names=("model", "k"))
results_knn = pd.DataFrame(
    index=index,
    columns=[
        "**Recall class 1**",
        "**F1 class 1**",
        "Accuracy",
        "F1 Macro",
        "Precission Macro",
        "Recall Macro",
    ],
)

k_values = range(1, 50, 2)

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn = knn.fit(X_train, y_train)
    y_pred = knn.predict(X_val)
    results_knn.loc[("KNN", k), :] = compute_metrics(y_val, y_pred)

results_knn.sort_values(by="**Recall class 1**", ascending=False)

According to our metrics we will chose k = 49.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=49)
knn = knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_val)
results_heart.loc["KNN-49", :] = compute_metrics(y_val, y_pred_knn)
results_heart

### Naive Bayes

Our final model, now we are going to use Naive Bayes in two different ways. First we are going to use gaussian naive bayes, asuming our variables are numerical. This is not fully true, as some of them are categorical variables transformed with a one-hot encoding. 

In the second way we will try to take advantage of these categorical variables by using to different naive bayes model, one categorical for the categorical variables and one gaussian for the numerical. As it is a probabilistic classifier we can use probabilities from both classifiers to predict our samples.  

In [ ]:
gaussian_nb = GaussianNB()
gaussian_nb = gaussian_nb.fit(X_train, y_train)
y_pred_gnb = gaussian_nb.predict(X_val)

results_heart.loc["Gaussian-NB", :] = compute_metrics(y_val, y_pred_gnb)
results_heart

In [ ]:
def preprocessing_only_numerical(X, scaler=None):
    # We scale the numerical columns
    if scaler is None:
        # We only want the scaler to fit the train data
        scaler = MinMaxScaler()
        X.loc[:, numerical_cols] = scaler.fit_transform(X[numerical_cols])
    else:
        X.loc[:, numerical_cols] = scaler.transform(X[numerical_cols])
    return X, scaler


# We need to load again our data so we can pre-process it differently
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=43
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.33, random_state=42
)


X_train_numerical, scaler = preprocessing_only_numerical(X_train[numerical_cols])
X_train_categorical = X_train[categorical_cols]

X_val_numerical, _ = preprocessing_only_numerical(X_val[numerical_cols], scaler)
X_val_categorical = X_val[categorical_cols]

X_test_numerical, _ = preprocessing_only_numerical(X_test[numerical_cols], scaler)
X_test_categorical = X_test[categorical_cols]

In [ ]:
gaussian_nb = GaussianNB()
gaussian_nb = gaussian_nb.fit(X_train_numerical, y_train)
y_pred_gnb_num = gaussian_nb.predict(X_val_numerical)

results_heart.loc["Gaussian-NB-only-numerical", :] = compute_metrics(
    y_val, y_pred_gnb_num
)


cat_nb = CategoricalNB()
cat_nb = cat_nb.fit(X_train_categorical, y_train)
y_pred_gnb_cat = cat_nb.predict(X_val_categorical)

results_heart.loc["Gaussian-NB-only-categorical", :] = compute_metrics(
    y_val, y_pred_gnb_cat
)

results_heart

Now we are going to use our probabilistic classifiers trick, we can use the probabilities prediction together by multiplying them, sometimes this kind of aggregation can improve the results of the two independent models. 

We can see on our results that this is not the case. For some reason our only numerical naive bayes is not working super well and it looks like it is affecting the aggregated results.

In [ ]:
combined_prediction_proba = cat_nb.predict_proba(
    X_val_categorical
) * gaussian_nb.predict_proba(X_val_numerical)

combined_prediction = np.argmax(combined_prediction_proba, axis=1)

results_heart.loc["Combined-NB", :] = compute_metrics(y_val, combined_prediction)
results_heart

Now there is a last probability trick that we can use. As we want to be completely sure that our model is predicting 0 only when the patient is not diseased, instead of selecting the class with the maximum probability value, we can select class 0 only when the combined probability of class 0 is greater than 0.6.   

In [ ]:
def only_cero_when_you_are_sure(x):
    if x[0] > 0.6:
        return 0
    else:
        return 1


combined_pred_tuned = pd.DataFrame(combined_prediction_proba).apply(
    only_cero_when_you_are_sure, axis=1
)

results_heart.loc["Combined-NB-tuned", :] = compute_metrics(y_val, combined_pred_tuned)

results_heart.sort_values(by="**Recall class 1**", ascending=False)

### Logistic regression

The last model we are going to compare is (regularized) Logistic regression. We are going to optimize its regularization parameter as in the other models, by picking the one that maximizes recall on class 1 on the validation set.

In [ ]:
index = pd.MultiIndex.from_arrays([[], []], names=("model", "C"))
results_logreg = pd.DataFrame(
    index=index,
    columns=[
        "**Recall class 1**",
        "**F1 class 1**",
        "Accuracy",
        "F1 Macro",
        "Precission Macro",
        "Recall Macro",
    ],
)

lambdas = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]

for l in lambdas:
    C = 1 / l
    logreg = LogisticRegression(C=C)
    logreg = logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_val)
    results_logreg.loc[("LogReg", C), :] = compute_metrics(y_val, y_pred)

results_logreg.sort_values(by="**Recall class 1**", ascending=False)

It seems the best logistic regression model according to recall on class 1 is with regularization parameter C=0.1. Let us use this model and compare it to the others.

In [ ]:
logreg = LogisticRegression(C=0.1).fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_val)
results_heart.loc["LogReg-0.1", :] = compute_metrics(y_val, y_pred_logreg)

results_heart.sort_values(by="**Recall class 1**", ascending=False)

This way we are obtaining the best model in terms of recall of the class 1. Notice that this model is not the best in terms of the F1 of class 1 nor the F1 Macro.  If we check our confusion matrix we can see that this model has predicted almost all patients correctly at the cost of having worse performance on the other class. 

In this specific context we wanted to absolutly prioritize avoiding misclassifying class 1. So our best choice would be the **Combined Naive Bayes Tuned**, if our criteria was different or the importance of missclasifying was more balanced between the classes the best model would be **LDA**.

In [ ]:
print("Combined Naive Bayes Tuned")
confusion(y_val, combined_pred_tuned)

print("LDA")
confusion(y_val, pd.Series(y_pred_lda))

print("QDA")
confusion(y_val, pd.Series(y_pred_qda))

print("Logistic Regression")
confusion(y_val, pd.Series(y_pred_logreg))

### See generalization performance

Now that we have decided our model. As there are no more decisions to make we can now check its generalization performance. To ensure that this generalization performance is the closest posible to the performance we would obtain on new data, we are going to use our test partition to evaluate it. 

This time we can see the confusion matrix, as well as the classification report for the test set. This will give us the most detailed possible information about our results.

These results are quite similar to the validation ones. 

In [ ]:
combined_prediction_proba_test = cat_nb.predict_proba(
    X_test_categorical
) * gaussian_nb.predict_proba(X_test_numerical)
combined_pred_tuned_test = pd.DataFrame(combined_prediction_proba_test).apply(
    only_cero_when_you_are_sure, axis=1
)

confusion(y_test, pd.Series(combined_pred_tuned_test))

print(classification_report(y_test, combined_pred_tuned_test))